In [1]:
import pandas as pd
from urllib.parse import urlparse
import re
import spacy

In [2]:
botify_dat = pd.read_csv('./botify_200codes_09292022.csv')
botify_dat = botify_dat.dropna(subset = ['First H1'])
botify_dat = botify_dat.drop_duplicates()

In [3]:
botify_dat.shape

(220331, 15)

In [4]:
botify_dat['First H1'].nunique()

63072

In [5]:
botify_dat['cleaned_First_H1'] = botify_dat['First H1'].apply(lambda x: re.sub(' +', ' ', x.lower()))

In [6]:
botify_dat['cleaned_First_H1'].nunique()

62880

In [7]:
# botify_dat.groupby('cleaned_First_H1')['First H1'].value_counts()

In [8]:
botify_dat['info_cnt'] = botify_dat[['Product Id', 'Breadcrumb Depth 1', 'Breadcrumb Depth 2',
       'Breadcrumb Depth 3', 'Breadcrumb Depth 4', 'Breadcrumb Depth 5',
       'Breadcrumb Depth 6']].apply(lambda x: x.isna()).sum(axis = 1)

In [9]:
unique_h1s = botify_dat.sort_values(['info_cnt', 'Total Count of Products'], ascending = [True, False]).drop_duplicates('First H1', ignore_index = True)

In [10]:
unique_h1s.shape

(63072, 17)

In [11]:
unique_h1s['Full_URL_netloc'] = unique_h1s['Full URL'].apply(lambda x: urlparse(x).netloc if isinstance(x, str) else x)

In [12]:
unique_h1s['Full_URL_netloc'].value_counts()

www.chewy.com    63072
Name: Full_URL_netloc, dtype: int64

In [13]:
unique_h1s['Full_URL2'] = unique_h1s['Full URL'].apply(lambda x:x.strip('/').split('/') if isinstance(x, str) else x)

In [14]:
unique_h1s['Full_URL2_types'] = unique_h1s['Full_URL2'].apply(lambda x: [w.strip() for w in x if len(str(w).strip()) <= 5 and len(str(w).strip()) > 0] if isinstance(x, list) else x)

In [15]:
unique_h1s['Full_URL2_types'].value_counts(dropna = False).head(5)

[dp]           46503
[f]            10988
[b]             2513
[dp, 37491]        2
[dp, 29861]        2
Name: Full_URL2_types, dtype: int64

In [16]:
unique_h1s['dp_page'] = unique_h1s['Full_URL2_types'].apply(lambda x: isinstance(x, list) and 'dp' in x)
unique_h1s['b_page'] = unique_h1s['Full_URL2_types'].apply(lambda x: isinstance(x, list) and 'b' in x)
unique_h1s['f_page'] = unique_h1s['Full_URL2_types'].apply(lambda x: isinstance(x, list) and 'f' in x)

In [17]:
for idx, vle in unique_h1s.iterrows():
    if vle['dp_page']:
        unique_h1s.loc[idx, 'page_type'] = 'dp'
    if vle['b_page']:
        unique_h1s.loc[idx, 'page_type'] = 'b'
    if vle['f_page']:
        unique_h1s.loc[idx, 'page_type'] = 'f'

In [18]:
unique_h1s['page_type'].value_counts(dropna = False)

dp    49571
f     10988
b      2513
Name: page_type, dtype: int64

In [19]:
unique_h1s[unique_h1s['dp_page'] == True]['Product Id'].value_counts(dropna = False).head(20)

NaN           23
sku:605854     2
sku:638686     2
sku:239934     2
sku:373349     2
sku:362760     2
sku:269326     2
sku:342232     2
sku:286579     2
sku:296309     2
sku:213106     2
sku:306527     2
sku:271925     2
sku:298937     2
sku:345640     2
sku:605406     2
sku:638574     2
sku:310597     2
sku:363585     2
sku:88027      2
Name: Product Id, dtype: int64

In [20]:
unique_h1s[(unique_h1s['dp_page'] == True) & (unique_h1s['Product Id'] == 'sku:605854')][['First H1']].value_counts() #extra spaces

First H1                                                                                                                                                             
Bundle: Made by Nacho Chicken, Duck & Quail Recipe With Freeze-Dried Chicken Liver +  Salmon, Whitefish & Pumpkin Recipe With Freeze-Dried Chicken Liver Dry Cat Food    1
Bundle: Made by Nacho Chicken, Duck & Quail Recipe With Freeze-Dried Chicken Liver + Salmon, Whitefish & Pumpkin Recipe With Freeze-Dried Chicken Liver Dry Cat Food     1
dtype: int64

In [21]:
unique_h1s[(unique_h1s['dp_page'] == True) & (unique_h1s['Product Id'] == 'sku:605854')][['cleaned_First_H1']].value_counts()

cleaned_First_H1                                                                                                                                                    
bundle: made by nacho chicken, duck & quail recipe with freeze-dried chicken liver + salmon, whitefish & pumpkin recipe with freeze-dried chicken liver dry cat food    2
dtype: int64

In [22]:
unique_h1s[unique_h1s['f_page'] == True]['Total Count of Products'].value_counts(dropna = False).head(20)

NaN     862
3.0     819
2.0     782
4.0     720
5.0     609
6.0     531
7.0     398
8.0     383
10.0    326
9.0     290
11.0    227
13.0    219
12.0    209
14.0    185
15.0    174
17.0    164
16.0    147
18.0    137
19.0    125
22.0    116
Name: Total Count of Products, dtype: int64

In [23]:
unique_h1s[unique_h1s['b_page'] == True]['Total Count of Products'].value_counts(dropna = False).head(20)

9.0     53
18.0    46
7.0     45
11.0    44
3.0     42
15.0    39
8.0     39
12.0    37
13.0    36
5.0     35
17.0    35
24.0    35
22.0    35
10.0    33
14.0    33
16.0    31
26.0    31
19.0    30
4.0     30
21.0    30
Name: Total Count of Products, dtype: int64

In [24]:
unique_h1s['First H1']

0        Living World Blue Ergonomic Small Pet Dish, Small
1        Living World Green Ergonomic Small Pet Dish, S...
2        Kaytee Hay-N-Food Bin with Quick Locks Small A...
3        Lixit Baby Small Animal Bottle Nursing Kit, 4-...
4                       Lixit Small Animal Top Fill Bottle
                               ...                        
63067                                             Equidone
63068                                            Ruff Hero
63069                                            ShieldTec
63070                                           CRB Palace
63071                                     Canine Creations
Name: First H1, Length: 63072, dtype: object

In [25]:
%%time
nlp = spacy.load('en_core_web_sm')
unique_h1s['First_H1_lemmatized'] = unique_h1s['cleaned_First_H1'].apply(lambda x: ' '.join([token.lemma_ if token.text != 'id' else token.text for token in nlp(x)]))

Wall time: 3min 34s


In [26]:
unique_h1s['cleaned_First_H1'].nunique()

62880

In [27]:
unique_h1s['First_H1_lemmatized'].nunique()

62858

In [28]:
unique_h1s.reset_index(drop = True).to_csv('./unique_h1s.csv', index = False)

In [29]:
unique_h1s = pd.read_csv('unique_h1s.csv')

C:\Users\ZachXiong\AppData\Local\Temp\ipykernel_21644\4185742324.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  unique_h1s = pd.read_csv('unique_h1s.csv')


In [30]:
unique_h1s[unique_h1s['Breadcrumb Depth 1'].isna()]['page_type'].value_counts()

f     2173
dp     358
b        5
Name: page_type, dtype: int64

In [31]:
unique_h1s[(unique_h1s['page_type'] == 'b') & (unique_h1s['Breadcrumb Depth 1'].isna())][['First H1', 'Breadcrumb Depth 1']].head(20)

,First H1,Breadcrumb Depth 1
62603,Cat Dental Care,NaN
62604,Small Pet Shop,NaN
62605,Autoship & Save,NaN
62705,Bird Shop,NaN
62854,Easter Shop,NaN
